
In this file, i will be buidling and training a machine learning pipeline to recognize if a person have mental health problems and making poisonous comments from texts they wrote and said, so it can help mental health doctors and expets in their diagnosis and tests in any medical company and institute


# Importing libraires, uploading dataset and data exploration

In [ ]:
# here i will import libraries used for loading datasets
import pandas as pd
# doing data split
import sklearn.model_selection
# Libraries for NLP preprocess functions and download the resources
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


I will load the dataset, and see how many lines and columns it have

In [ ]:
df = pd.read_csv("mental_health.csv")
df

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1
...,...,...
27972,posting everyday people stop caring religion ...,0
27973,okay definetly need hear guys opinion ive pret...,0
27974,cant get dog think ill kill myselfthe last thi...,1
27975,whats point princess bridei really think like ...,1


here i will see if there is a big difference between how many times a comment was labeled to indicate mental health or not, having big difference means it is better to balance the data

In [ ]:
c1 = (df["label"] == 1).sum()
c2 = (df["label"] == 0).sum()
print(c1)
print(c2)

13838
14139


# Preprocess and encoding

I will look for any empty cells in the dataset and delete if any exist

In [ ]:
dfnull = df.isna().sum()
dfnull

text     0
label    0
dtype: int64

Here i will look for any duplicate rows, and delete them if they exists

In [ ]:
dfduplicate = df.duplicated().sum()
dfduplicate

5

In [ ]:
df = df.drop_duplicates()

here i will define a function that tokenize words ,remove stop words, and lemmatize the words (transform the words to their base)

In [ ]:
def preprocessdata(texts):
    W = word_tokenize(texts.lower())
    W = [token for token in W if token not in stop_words]
    lem = WordNetLemmatizer()
    W = [lem.lemmatize(token) for token in W]
    return ' '.join(W)

Now i will use the tfidf vectorizer to, i will use the function i defined earlier with the tokenizer hyperpaprameter
then give a different range for the n grams

set sublinear_tf to false so that it lower/mitigate impact of frequeebt terms

and make use_idf to false, it will not give more weight to terms that are not repeated much

i will set "token_pattern" to none to remove warnings that it's not used

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=preprocessdata, lowercase=False, sublinear_tf=False, ngram_range=(1,3), min_df=10, token_pattern=None)
vectorizedtext = vectorizer.fit_transform(df['text'])

In [ ]:
# here i will see if the changes happened to the column
df2 = (vectorizedtext, df['label'])
df2
# and it shows that it have the column label
# with the sparse matrix we got from vectorizing the text column

(<27972x7544 sparse matrix of type '<class 'numpy.float64'>'
 	with 11382567 stored elements in Compressed Sparse Row format>,
 0        0
 1        1
 2        0
 3        1
 4        1
         ..
 27972    0
 27973    0
 27974    1
 27975    1
 27976    0
 Name: label, Length: 27972, dtype: int64)

after all the preprocessing steps, i will split the data, to train and test and seperate the labels

In [ ]:
x_train,x_test, y_train , y_test = sklearn.model_selection.train_test_split(vectorizedtext , df['label'], test_size=0.15)

In [ ]:
# here i will import all the libraries i need to train the model
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
# to see the results of applying the model on the test data
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
# and later i will pick the best using GridsearchCV

In [ ]:
# i give names to the imported machine learning libraries
T = DecisionTreeClassifier()
S = SVC()
NB = MultinomialNB()
LR = LogisticRegression()

Adjusting the hyperpaprameters for the models

In [ ]:
# for the decision tree, i will try gini and entropy function to measure quality of the tree spliting
# i will different tree depth, and numbers required for a split to avoid overfitting
# put some values for min_impurity_decrease to put more restriction for splits in the tree
T_param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [3, 5, 10],
                'min_samples_split': [2, 4], 'min_impurity_decrease': [1,2]}

# For svm, i will keep on linear because we don't many columns so no need to waste computing time with rbf
S_param_grid = {'kernel': ['linear']}

# for logistic regression i will try using all processors and not
LR_param_grid = { 'n_jobs': [None, -1]}

# for MultinomialNB i will see if any difference will happen when learning with prior probabilities or not
NB_param_grid = {'fit_prior': [True,False]}

In [ ]:
T_gs = GridSearchCV(T, T_param_grid, scoring='accuracy')
T_gs.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 10],
                         'min_impurity_decrease': [1, 2],
                         'min_samples_split': [2, 4]},
             scoring='accuracy')

In [ ]:
S_gs = GridSearchCV(S, S_param_grid, scoring='accuracy')
S_gs.fit(x_train, y_train)

GridSearchCV(estimator=SVC(), param_grid={'kernel': ['linear']},
             scoring='accuracy')

In [ ]:
LR_gs = GridSearchCV(LR, LR_param_grid, scoring='accuracy')
LR_gs.fit(x_train, y_train)

GridSearchCV(estimator=LogisticRegression(), param_grid={'n_jobs': [None, -1]},
             scoring='accuracy')

In [ ]:
NB_gs = GridSearchCV(NB, NB_param_grid, scoring='accuracy')
NB_gs.fit(x_train, y_train)

GridSearchCV(estimator=MultinomialNB(), param_grid={'fit_prior': [True, False]},
             scoring='accuracy')

and now i want to pick the best model

In [ ]:
best_model =  T_gs.best_estimator_ if T_gs.best_score_ > LR_gs.best_score_ and T_gs.best_score_ > S_gs.best_score_ and T_gs.best_score_ > NB_gs.best_score_ else S_gs.best_estimator_ if S_gs.best_score_ > LR_gs.best_score_ and S_gs.best_score_ > NB_gs.best_score_ else LR_gs.best_estimator_ if LR_gs.best_score_ > NB_gs.best_score_ else NB_gs.best_estimator_
best_model.fit(x_train, y_train)

SVC(kernel='linear')

In [ ]:
# and here i will see the result of the model on the test set
# and get the results for accuracy, f1, precsion, recall
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("accuracy:", accuracy)
# and also check the confusion matrix
print('confusion matrix:\n', confusion_matrix(y_test, y_pred))
print("FI:", f1)
print("Precision:", precision)
print("recall:", recall)

accuracy: 0.9251668255481411
confusion matrix:
 [[1936  151]
 [ 163 1946]]
FI: 0.9253447456015217
Precision: 0.9279923700524559
recall: 0.9227121858700806


Discussion:


We got some good results at the end of this pipeline, but there are few things to consider:


The accuracy number is very good, when we see the recall number, it suggests that we are able to deal good with comments that are poisonous and people saying them have mental issues behind them and labeled as such in the dataset, precision shows that most of the positive predictions the model made are true, F1 score being close to 1, show us that the model phave great performance


as for the confusion matrix, we can still see some instances that were predicted wrong, and it's bit more for instances that are predicted negative but are actually positive.


taking into account that there is not much difference between the size of positive and negative data/instances (with negative/'0' counts slightly more), we can conclude that the results are good.


for later improvement, we can consider using word2vec instead of tfidf, it could be more time consuming, but might capture the meaning better, and it's possible to use deep learning algorithms instead of traditional models.

Dataset Refrencing:

Namdari, R. (2022) Mental health corpus, Kaggle. Available at: https://www.kaggle.com/datasets/reihanenamdari/mental-health-corpus (Accessed: 27 June 2023).


(Note: the dataset was updated 6 months ago as of the date of access)